# 毕业论文`GRAD`
## 建立多元线性回归模型，定量评估影响

*`Evan`*\
*`2023-11-16`*
---

In [1]:
import numpy as np
import pandas as pd
import xarray as xr

import sys
sys.path.append('../../src/')
from namelist import *
from RandomForest import write_nc_to_df

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

In [2]:
lowyears  = [2014,2015,2016]
highyears = [2019,2021,2022]
month     = 'Sep'

# 设置从mcip和chem文件中读取的变量
mcip_variants = ['SFC_TMP', 'SOL_RAD', 'RH', 'PRES', 
                 'WSPD10', 'WDIR10','CloudFRAC','PBLH']
chem_variants = ['O3','NO2', 'VOC', 'PM25', 'ISOP']

# 设置输入rf模型的自变量列表和目标变量
variants = ['SFC_TMP', 'SOL_RAD', 'RH', 'PRES', 
            'WSPD10', 'WDIR10','CloudFRAC',
            'NO2', 'VOC', 'PM25', 'ISOP'] # 'PBLH',
target = 'O3'

In [3]:
dfs = write_nc_to_df(lowyears, month, processed_dir,
                     mcip_variants, chem_variants, variants, target,
                     nx=4,ny=4)

nx = 4, ny = 4
The process is expected to take up to 1.87 min, or 0.03 h
(0,0) --> 6.25 %
(1,0) --> 12.50 %
(2,0) --> 18.75 %
(3,0) --> 25.00 %
(0,1) --> 31.25 %
(1,1) --> 37.50 %
(2,1) --> 43.75 %
(3,1) --> 50.00 %
(0,2) --> 56.25 %
(1,2) --> 62.50 %
(2,2) --> 68.75 %
(3,2) --> 75.00 %
(0,3) --> 81.25 %
(1,3) --> 87.50 %
(2,3) --> 93.75 %
(3,3) --> 100.00 %


## 将`Dataframe`转为`Dataset`

In [34]:
dfs.keys()

dict_keys(['SFC_TMP', 'SOL_RAD', 'RH', 'PRES', 'WSPD10', 'WDIR10', 'CloudFRAC', 'NO2', 'VOC', 'PM25', 'ISOP', 'mse', 'r2'])

In [35]:
grid = xr.open_dataset(grid_d03)
lon = grid.LON.squeeze().values
lat = grid.LAT.squeeze().values

dataset = xr.Dataset(
    data_vars=dict(
        SFC_TMP   = (['y','x'],dfs['SFC_TMP'].values),
        SOL_RAD   = (['y','x'],dfs['SOL_RAD'].values),
        RH        = (['y','x'],dfs['RH'].values),
        PRES      = (['y','x'],dfs['PRES'].values),
        WSPD10    = (['y','x'],dfs['WSPD10'].values),
        WDIR10    = (['y','x'],dfs['WDIR10'].values),
        CloudFRAC = (['y','x'],dfs['CloudFRAC'].values),
        NO2       = (['y','x'],dfs['NO2'].values),
        VOC       = (['y','x'],dfs['VOC'].values),
        PM25      = (['y','x'],dfs['PM25'].values),
        ISOP      = (['y','x'],dfs['ISOP'].values),
        mse       = (['y','x'],dfs['mse'].values),
        r2        = (['y','x'],dfs['r2'].values),
    ),
    coords=dict(
        latitude = (['y','x'],lat),
        longitude = (['y','x'],lon),
    ),
)
dataset

<xarray.Dataset>
Dimensions:    (y: 110, x: 152)
Coordinates:
    latitude   (y, x) float32 21.54 21.54 21.54 21.55 ... 24.57 24.57 24.57
    longitude  (y, x) float32 111.1 111.2 111.2 111.2 ... 115.6 115.6 115.7
Dimensions without coordinates: y, x
Data variables: (12/13)
    SFC_TMP    (y, x) object 0.08998 0.08361 0.08391 0.05365 ... nan nan nan nan
    SOL_RAD    (y, x) object 0.1867 0.1792 0.1656 0.18 nan ... nan nan nan nan
    RH         (y, x) object 0.03993 0.02928 0.02791 0.02938 ... nan nan nan nan
    PRES       (y, x) object 0.0358 0.04866 0.05556 0.05635 ... nan nan nan nan
    WSPD10     (y, x) object 0.03471 0.02722 0.03437 0.03149 ... nan nan nan nan
    WDIR10     (y, x) object 0.07174 0.04895 0.03651 0.03309 ... nan nan nan nan
    ...         ...
    NO2        (y, x) object 0.0527 0.05373 0.04724 0.04537 ... nan nan nan nan
    VOC        (y, x) object 0.0539 0.05259 0.0469 0.04186 ... nan nan nan nan
    PM25       (y, x) object 0.3945 0.4384 0.4603 0.4885 nan ... nan nan nan nan
    ISOP       (y, x) object 0.01791 0.01806 0.0219 0.019 ... nan nan nan nan
    mse        (y, x) object 269.0 281.5 284.7 307.8 nan ... nan nan nan nan nan
    r2         (y, x) object 0.8264 0.8215 0.8213 0.8091 nan ... nan nan nan nan

In [36]:
dataset.to_netcdf('D:/Download/test.nc')